# Quick Start Guide - Azure AI Foundry

This notebook provides a hands-on introduction to Azure AI Foundry. You'll learn how to:
1. Initialize the AI Project client
2. List available models
3. Create a simple chat completion request
4. Create a basic AI agent
5. Handle basic error scenarios

## Prerequisites
- Completed environment setup from previous notebook
- Azure credentials configured
- **azure-ai-projects** package version 1.0.0b12 or greater (`azure-ai-projects>=1.0.0b12`)
- **Azure AI User role** assigned to your account for the Azure AI Foundry project
  - See [Azure AI Foundry RBAC documentation](https://learn.microsoft.com/en-us/azure/ai-foundry/concepts/rbac-azure-ai-foundry?pivots=fdp-project) for more details on role assignments

## Import Required Libraries and Setup

In the next cell, we'll:
1. Import the necessary Azure SDK libraries for authentication and AI Projects
2. Import standard Python libraries for environment variables and JSON handling
3. Initialize Azure credentials using DefaultAzureCredential
   - This will automatically use your logged-in Azure CLI credentials
   - Alternatively, it can use other authentication methods like environment variables or managed identity

## 🔐 Authentication Setup

Before running the next cell, make sure you're authenticated with Azure CLI. Run this command in your terminal:

```bash
az login --use-device-code
```

This will provide you with a device code and URL to authenticate in your browser, which is useful for:
- Remote development environments
- Systems without a default browser
- Corporate environments with strict security policies

After successful authentication, you can proceed with the notebook cells below.


In [ ]:
# Import required libraries
from azure.identity import DefaultAzureCredential, AzureCliCredential, InteractiveBrowserCredential, ChainedTokenCredential
from azure.ai.projects import AIProjectClient
import os
import json

# Initialize credentials with tenant-specific authentication (same as environment setup)
from dotenv import load_dotenv
load_dotenv()

# Get the correct tenant ID from environment
correct_tenant_id = os.getenv("TENANT_ID")
print(f"🔑 Using Tenant ID: {correct_tenant_id}")

# Create a credential chain with tenant-specific authentication
def create_credential_chain_with_tenant():
    """Create a robust credential chain for authentication with specific tenant"""
    try:
        # Try Azure CLI first with the specific tenant
        cli_credential = AzureCliCredential(tenant_id=correct_tenant_id)
        
        # Create a chained credential with fallbacks, all using the correct tenant
        credential_chain = ChainedTokenCredential(
            cli_credential,
            InteractiveBrowserCredential(tenant_id=correct_tenant_id)
        )
        
        return credential_chain
    except Exception as e:
        print(f"⚠️ Credential chain creation error: {e}")
        # Fallback to DefaultAzureCredential with tenant specified
        return DefaultAzureCredential(tenant_id=correct_tenant_id)

# Initialize credentials
try:
    credential = create_credential_chain_with_tenant()
    
    # Test the credential by getting a token for the correct tenant
    test_token = credential.get_token("https://management.azure.com/.default")
    print("✅ Successfully initialized Azure credentials with correct tenant!")
    
except Exception as e:
    print(f"❌ Credential initialization failed: {str(e)}")
    print(f"💡 Please run the authentication fix from the environment setup notebook first")
    credential = None

## Initialize AI Project Client

> **Note:** Before proceeding, ensure you:
> 1. Copy your `.env.example` file to `.env` from the root directory
> 2. Update the project connection string in your `.env` file
> 3. Have a Foundry Project already provisioned in Azure AI Foundry

You can find your project connection string in [Azure AI Foundry](https://ai.azure.com) under your project's settings:

<img src="images/proj-conn-string.png" alt="Project Connection String Location" width="600"/>


## Creating the AI Project Client

In the next cell, we'll create an AI Project client using the connection string from our `.env` file.
> **Note:** This example uses the synchronous client. For higher performance scenarios, you can also create an asynchronous client by importing `asyncio` and using the async methods from `AIProjectClient`.

The client will be used to:
- Connect to your Azure AI Project using the connection string
- Authenticate using Azure credentials
- Enable making inference requests to your deployed models


In [ ]:
from dotenv import load_dotenv
from pathlib import Path

# Load environment variables from root directory
notebook_path = Path().absolute()
# Navigate to the root directory (up 3 levels: from Labs/Connection with AOAI/ to root)
root_dir = notebook_path.parent.parent.parent
env_path = root_dir / '.env'

print(f"📁 Looking for .env file at: {env_path}")
load_dotenv(env_path)

try:
    # Get the project connection string (which is actually the endpoint URL)
    ai_foundry_project_endpoint = os.getenv("AI_FOUNDRY_PROJECT_ENDPOINT")
    if not ai_foundry_project_endpoint:
        raise ValueError("AI_FOUNDRY_PROJECT_ENDPOINT not found in environment variables")
    
    print(f"🔗 Project Connection: {ai_foundry_project_endpoint}")
    
    # Create AIProjectClient as done in environment_setup (consistent with other notebooks)
    client = AIProjectClient(
        credential=credential,
        endpoint=ai_foundry_project_endpoint
    )
    print("✓ Successfully initialized AIProjectClient")
except Exception as e:
    print(f"× Error initializing client: {str(e)}")
    print("💡 Tip: Make sure your AI_FOUNDRY_PROJECT_ENDPOINT is set in the .env file at the root directory")
    print(f"💡 Expected .env location: {env_path}")

## Create a Simple Completion
Let's try a basic completion request:

Now that we have an authenticated client, let's use it to make a chat completion request.
The code below demonstrates how to:
1. Get a ChatCompletionsClient from the azure-ai-inference package
2. Use it to make a simple completion request

We'll use the MODEL_DEPLOYMENT_NAME from our `.env` file, making it easy to switch between different
deployed models without changing code. This could be an Azure OpenAI model, Microsoft model, or other providers
that support chat completions.

> Note: Make sure you have the azure-ai-inference package installed (from requirements.txt or as mentioned in [README.md](../README.md#-quick-start))


In [ ]:
from azure.ai.inference.models import UserMessage

model_deployment_name = os.getenv("MODEL_DEPLOYMENT_NAME")

try:
    # Use the correct Azure AI Projects SDK pattern
    print("🔄 Getting OpenAI client from Azure AI Project...")
    print(f"🤖 Using model: {model_deployment_name}")
    
    # Get OpenAI client using the correct method
    openai_client = client.get_openai_client(api_version="2024-10-21")
    
    # Create chat completion using OpenAI client pattern
    response = openai_client.chat.completions.create(
        model=model_deployment_name,
        messages=[
            {"role": "system", "content": "You are a helpful health assistant"},
            {"role": "user", "content": "How to be healthy in one sentence?"}
        ]
    )
    
    print("✅ Successfully created chat completion!")
    print(f"🤖 Assistant: {response.choices[0].message.content}")
    
except Exception as e:
    print(f"❌ An error occurred: {str(e)}")
    print("💡 Troubleshooting tips:")
    print("  - Ensure your Azure AI Project has OpenAI connections configured")
    print("  - Verify your MODEL_DEPLOYMENT_NAME is correctly deployed")
    print("  - Check that you have proper permissions to access the model")
    print("  - Make sure you're using the latest azure-ai-projects SDK version")

## Create a simple Agent

Using AI Agent Service, we can create a simple agent to answer health related questions.

Let's explore Azure AI Agent Service, a powerful tool for building intelligent agents.

Azure AI Agent Service is a fully managed service that helps developers build, deploy, and scale AI agents
without managing infrastructure. It combines large language models with tools that allow agents to:
- Answer questions using RAG (Retrieval Augmented Generation)
- Perform actions through tool calling 
- Automate complex workflows

The code below demonstrates how to:
1. Create an agent with a code interpreter tool
2. Create a conversation thread
3. Send a message requesting BMI analysis 
4. Process the request and get results
5. Save any generated visualizations to local files

The agent will use the model specified in our .env file (MODEL_DEPLOYMENT_NAME) and will have access
to a code interpreter tool for creating visualizations. This showcases how agents can combine
natural language understanding with computational capabilities.

> **Note:** Generated visualizations will be saved as PNG files in the same folder as this notebook.
 



In [ ]:
from azure.ai.agents.models import CodeInterpreterTool

try:
    # Initialize the Code Interpreter Tool
    code_interpreter = CodeInterpreterTool()
    
    # Create an AI agent with the code interpreter tool
    agent = client.agents.create_agent(
        model=model_deployment_name,
        name="bmi-calculator",
        instructions=(
            "You are a health analyst who calculates BMI using US metrics (pounds, feet/inches). "
            "Use average US female measurements: 5'4\" (69 inches) and 130 pounds. "
            "Create a visualization showing where this BMI falls on the scale."
        ),
        tools=code_interpreter.definitions,
        tool_resources=code_interpreter.resources,
    )
    
    # Create a new conversation thread
    thread = client.agents.threads.create()
    
    # Create a message requesting BMI analysis and visualization
    message = client.agents.messages.create(
        thread_id=thread.id,
        role="user",
        content=(
            "Calculate BMI for an average US female (5'4\", 130 lbs). "
            "Create a visualization showing where this BMI falls on the standard BMI scale from 15 to 35. "
            "Include the standard BMI categories (Underweight, Normal, Overweight, Obese) in the visualization."
        )
    )
    
    # Process the request by creating and running a thread run
    run = client.agents.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
    
    # Get the agent's response
    messages = client.agents.messages.list(thread_id=thread.id)
    
    # Print the assistant's response and save any images
    for message in messages:
        if message.role == "assistant":
            # Print text content
            for content_item in message.content:
                if hasattr(content_item, 'text'):
                    print(f"🤖 Assistant: {content_item.text.value}")
            
            # Save any image files using the correct API
            for img in message.image_contents:
                file_id = img.image_file.file_id
                file_name = f"bmi_analysis_{file_id}.png"
                
                try:
                    # Use the correct file saving method
                    client.agents.files.save(file_id=file_id, file_name=file_name)
                    print(f"📊 Visualization saved as: {file_name}")
                    
                except Exception as file_error:
                    print(f"⚠️ Could not save file {file_id}: {file_error}")
                    print("💡 You can view the file in the Azure AI Foundry portal")
            break
    
    # Cleanup by deleting the agent
    client.agents.delete_agent(agent.id)
    
except Exception as e:
    print(f"An error occurred: {str(e)}")